Imports

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from queue import PriorityQueue
import torch
import os
import glob

Carrega MiDaS

In [ ]:
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
midas.to('cpu')
midas.eval()

Entrada pipeline

In [ ]:
transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

Diretorio das imagens

In [ ]:
image_dir = '..\\teste'
image_files = glob.glob(os.path.join(image_dir, '*'))

Usa a rede neural para aplicar profundidade na imagem

In [ ]:
for image_file in image_files:
    # Ler a imagem do diretório
    image_path = os.path.join(image_dir, image_file)
    frame = cv2.imread(image_path)

    # Trasnformar entrada para o MiDaS
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to('cpu')

    # Fazer predição
    with torch.no_grad():
        prediction = midas(imgbatch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze()

        output = prediction.cpu().numpy()

        # Normalizar a saída para valores entre 0 e 255
        output = (output - output.min()) / (output.max() - output.min())
        output = (255 * output).astype(np.uint8)

    plt.imshow(output, cmap='gray')
    plt.pause(0.00001)

    # Salvar a imagem de saída
    output_filename = os.path.splitext(image_file)[0] + "_map.png"
    output_path = os.path.join(image_dir, output_filename)
    cv2.imwrite(output_path, output)
    print(f'Saved {output_path}')
    print(image_file)

Carrega a imagem

In [ ]:
img = cv2.imread('../teste/figuraCinco_map.png',
                 cv2.IMREAD_GRAYSCALE)

deubom = False

if img is None:
    print("Imagem não carregada.")
else:
    deubom = True

In [ ]:
linha_central = 580 #linha principal do espectro de intensidade do pixel

if deubom:
    x = img[linha_central, :]

    window_size = 13 
    x = np.convolve(x, np.ones(window_size) / window_size, 'same')
    plt.plot(x)

    # segunda derivada da linha central da imagem
    dx = np.diff(x)
    dx[:20] = 0
    dx[-20:]= 0
    
    plt.figure()
    plt.plot(dx)

    # picos positivos
    pks_positivos = []
    y_values_positivos = []
    while (np.max(dx) > 2.5) and len(pks_positivos) < 10:
        pos = np.argmax(dx)
        pks_positivos.append(pos)
        y_values_positivos.append(dx[pos])
        lo, hi = max(0, pos - 50), min(len(x), pos + 50)
        dx[lo:hi] = 0
    print("Picos Positivos:", pks_positivos)

    # picos negativos
    dx_negativo = -dx
    pks_negativos = []
    while (np.max(dx_negativo) > 2.5) and len(pks_negativos) < 10:
        pos = np.argmax(dx_negativo)
        pks_negativos.append(pos)
        lo, hi = max(0, pos - 50), min(len(x), pos + 50)
        dx_negativo[lo:hi] = 0
    print("Picos Negativos:", pks_negativos)

    # Crie a tupla de obstáculos
    if len(pks_positivos) > 0 or len(pks_negativos) > 0:

        if len(pks_positivos) == 0:
            pks_positivos.append(0)

        if len(pks_negativos) == 0:
            pks_negativos.append(len(dx))

        if pks_negativos[0] < pks_positivos[0]:
            pks_positivos = [0] + pks_positivos

        if pks_negativos[-1] < pks_positivos[-1]:
            pks_negativos = pks_negativos + [len(dx)]

        
        if len(pks_positivos) != len(pks_negativos):
            print("erro", pks_positivos, pks_negativos, len(dx))
        
    mean_x = [(a+b)//2 for a, b in zip(pks_positivos, pks_negativos)]
    y_values_positivos = [3*x[c] for c in mean_x]
    obstaculos = list(
        zip(pks_positivos, pks_negativos, y_values_positivos))
    print("Obstáculos:", obstaculos)

    # Filtrar obstáculos com base nos critérios
    obstaculos_filtrados = [
        obstaculo for obstaculo in obstaculos if obstaculo[0] >= 5 or obstaculo[1] <= 1595]
    print("Obstáculos filtrados:", obstaculos_filtrados)

   # Define limites mínimos para altura e diferença entre início e fim de obstáculos
    limite_altura_minima = 126
    limite_diferenca_x = 10

    # Define as dimensões do mapa aéreo (ajuste conforme necessário)
    largura_mapa = 1600
    altura_mapa = 899

    # Cria uma matriz para representar o mapa aéreo
    mapa_aereo = np.zeros((altura_mapa, largura_mapa))

    # Preenche o mapa aéreo com os obstáculos significativos
    for obstaculo in obstaculos_filtrados:
        inicio_x, fim_x, altura_y = obstaculo
        altura_y = int(altura_y)
        inicio_x = max(0, inicio_x)
        fim_x = min(largura_mapa - 1, fim_x)
        if altura_y >= altura_mapa:
            altura_y = altura_mapa - 1
        for x in range(inicio_x, fim_x + 1):
            mapa_aereo[altura_y, x] = 1

    plt.figure()
    plt.imshow(mapa_aereo, cmap='gray')
    plt.show()

    nome_arquivo_saida = 'mapa_aereoDois.png'
    cv2.imwrite(nome_arquivo_saida, mapa_aereo * 255)
    print(f"Mapa aéreo salvo como '{nome_arquivo_saida}'")

Carrega o mapa aereo

In [ ]:
aereoMap = cv2.imread('mapa_aereoDois.png', cv2.IMREAD_GRAYSCALE)

In [ ]:
# dilatação dos obstáculos
kernel = np.ones((200, 200), np.uint8)
img_dilated = cv2.dilate(aereoMap, kernel, iterations=1)

# centro da base do eixo X
start_node = (aereoMap.shape[0] - 1, aereoMap.shape[1] // 2)

# destino
goal_node = (50, 800)

Heuristica

In [ ]:
def heuristic(node, goal, img):  # distância euclidiana
    penalty = 1.0
    distance = np.sqrt((node[0] - goal[0]) ** 2 + (node[1] - goal[1]) ** 2)
    penalty_term = penalty * img[node[0], node[1]] / 255.0
    return distance + penalty_term

In [ ]:
def is_valid(node, img):  # verifica limites do mapa e se não é um obstáculo
    if (
        0 <= node[0] < img.shape[0] and
        0 <= node[1] < img.shape[1] and
        img[node[0], node[1]] == 0
    ):
        return True
    return False

Algoritmo A Estrela

In [ ]:
def astar(img, start, goal):
    open_set = PriorityQueue()
    open_set.put((0, start))
    came_from = {}
    g_score = {node: float('inf') for node in np.ndindex(img.shape)}
    g_score[start] = 0
    f_score = {node: float('inf') for node in np.ndindex(img.shape)}
    f_score[start] = heuristic(start, goal, img)

    while not open_set.empty():
        _, current = open_set.get()

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for neighbor in [(current[0] - 1, current[1]), (current[0] + 1, current[1]),
                         (current[0], current[1] - 1), (current[0], current[1] + 1)]:
            if is_valid(neighbor, img):
                tentative_g_score = g_score[current] + 1
                if tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + \
                        heuristic(neighbor, goal, img)
                    open_set.put((f_score[neighbor], neighbor))

    return None

In [ ]:
path = astar(img_dilated, start_node, goal_node)

Desenha caminho

In [ ]:
if path:
    path_points = np.array(path)

    plt.plot(path_points[:, 1], path_points[:, 0], 'r-', linewidth=2)

    n = 70
    selected_points = path_points[::n]

    plt.plot(selected_points[:, 1], selected_points[:, 0], 'go', markersize=3)

plt.imshow(aereoMap, cmap='gray')
plt.title('Mapa Aéreo com Rota Encontrada')
plt.xlabel('Eixo X')
plt.ylabel('Eixo Y')
plt.show()